In [8]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from difflib import SequenceMatcher
import Levenshtein

# Read recipe inputs
CALCULATED_DRAW_DOWNS = dataiku.Dataset("CALCULATED_DRAW_DOWNS")
CALCULATED_DRAW_DOWNS_df = CALCULATED_DRAW_DOWNS.get_dataframe()

CALCULATED_DRAW_UPS = dataiku.Dataset("CALCULATED_DRAW_UPS")
CALCULATED_DRAW_UPS_df = CALCULATED_DRAW_UPS.get_dataframe()

COMMON_WORDS = dataiku.Dataset("NAFCUSTOMER_COMMON_WORDS_IN_NAMES")
COMMON_WORDS_df = COMMON_WORDS.get_dataframe()

In [9]:
def date_tz_naive(pd_s):
    return pd.to_datetime(pd_s).apply(lambda x:x.tz_localize(None))

In [19]:
# consider customers with the card threshold or more
# set this too low and the running time will balloon
# recommend: 10 or higher

card_cut_off_threshold = 0

df_down_full = CALCULATED_DRAW_DOWNS_df[CALCULATED_DRAW_DOWNS_df.ACTIVE_CARD_MAX>card_cut_off_threshold].copy()
df_up_full = CALCULATED_DRAW_UPS_df[CALCULATED_DRAW_UPS_df.ACTIVE_CARD_MAX>card_cut_off_threshold].copy()

df_down_full.DRAW_DOWN_DATE = date_tz_naive(df_down_full['DRAW_DOWN_DATE'])
df_up_full.DRAW_UP_DATE = date_tz_naive(df_up_full['DRAW_UP_DATE'])

df_up_full.dropna(subset=['DRAW_UP_DATE'], inplace=True)

df_common = COMMON_WORDS_df

df_down_full.sort_values(['CUSTOMER'], inplace=True)
df_up_full.sort_values(['CUSTOMER'], inplace=True)
df_common.sort_values(['WORD'], inplace=True)

print(len(df_down_full), "draw downs full")
print(len(df_up_full), "draw ups full")
print(len(df_common), "common words")

186871 draw downs full
330570 draw ups full
2173 common words


In [20]:
import string

_common_words = df_common.WORD.unique()
print(len(_common_words), "screening against common words")

class Draw_Down_Customer:

    def __init__(self, name, draw_down_date, active_card_max):

        self.CUSTOMER = name
        self.DRAW_DOWN_DATE = draw_down_date
        self.ACTIVE_CARD_MAX = active_card_max

        self.MATCHING_CUSTOMERS = []
        self.PERCENT_DIFFERENCE = []
        self.DAYS_DIFFERENCE = []
        self.DRAW_UP_DATE = []

        # remove punctuation
        c_str = name.translate(str.maketrans('', '', string.punctuation))

        f = c_str.split()
        self.WORD_LIST = []
        for w in f:
            if w not in _common_words:
                self.WORD_LIST.append(w)

    def Match_Draw_Up_Customer(self, name, draw_up_date, active_card_max):

        if (self.CUSTOMER == name):
            # exact match, already captured
            return

        c_str = name.translate(str.maketrans('', '', string.punctuation))

        f = c_str.split()

        check_list = []
        for w in f:
            if (w not in _common_words) and (len(w)>1) and (not w.isnumeric()):
                check_list.append(w)

        percent_diff = round((abs(self.ACTIVE_CARD_MAX - active_card_max) / ((self.ACTIVE_CARD_MAX+active_card_max)/2)),2)

        #date_format = "%Y-%m-%d"
        #d1_date = datetime.strptime(draw_up_date.astype(str), date_format)
        #d2_date = datetime.strptime(self.DRAW_DOWN_DATE.astype(str), date_format)

        delta_between_drop_and_rise = round(abs((draw_up_date-self.DRAW_DOWN_DATE).days)/30.,0)

        for w_to_check in check_list:
            for w in self.WORD_LIST:
                if w_to_check==w:

                    if not name in(self.MATCHING_CUSTOMERS) and(delta_between_drop_and_rise<=4)and(percent_diff<=0.5) :
                        self.MATCHING_CUSTOMERS.append(name)
                        self.PERCENT_DIFFERENCE.append(percent_diff)
                        self.DAYS_DIFFERENCE.append(delta_between_drop_and_rise)
                        self.DRAW_UP_DATE.append(draw_up_date)
                        break;

2173 screening against common words


In [21]:
import time

def do_save_log(_matching_process_log_time, _matching_process_log_event):

    df_matching_log = pd.DataFrame(_matching_process_log_time)
    if len(df_matching_log)>0:

        df_matching_log.columns = ['LOG_TIME']
        df_matching_log['LOG_EVENT'] = _matching_process_log_event

        MATCHING_PROCESS_LOG_V_df = df_matching_log
        MATCHING_PROCESS_LOG_V = dataiku.Dataset("MATCHING_PROCESS_LOG_V")
        MATCHING_PROCESS_LOG_V.write_with_schema(MATCHING_PROCESS_LOG_V_df)

        print()

def do_save_direct_matches(_direct_customer, _direct_match, _direct_draw_up_date):

    df_matches = pd.DataFrame(_direct_customer)
    if len(df_matches)>0:

        print()
        print("saving", len(df_matches), "1-1 matching records")
        print()

        df_matches.columns = ['CUSTOMER']
        df_matches["MATCH_CUSTOMER"] = _direct_match
        df_matches["DRAW_UP_DATE"] = _direct_draw_up_date

        MATCHES_1_TO_1_STAGING_V_df = df_matches
        MATCHES_1_TO_1_STAGING_V = dataiku.Dataset("MATCHES_1_TO_1_STAGING_V")
        MATCHES_1_TO_1_STAGING_V.write_with_schema(MATCHES_1_TO_1_STAGING_V_df)

        print()

def do_save_multiple_matches(_multiple_customer, _multiple_matches, _multiple_drop_dates):

    df_multiple_matches = pd.DataFrame(_multiple_customer)

    if len(df_multiple_matches)>0:

        print()
        print("saving", len(df_multiple_matches), "1-n matching records")
        print()

        df_multiple_matches.columns = ['CUSTOMER']
        df_multiple_matches["MATCH_CUSTOMER"] = _multiple_matches
        df_multiple_matches["DRAW_UP_DATE"] = _multiple_drop_dates

        MATCHES_1_TO_N_STAGING_V_df = df_multiple_matches
        MATCHES_1_TO_N_STAGING_V = dataiku.Dataset("MATCHES_1_TO_N_STAGING_V")
        MATCHES_1_TO_N_STAGING_V.write_with_schema(MATCHES_1_TO_N_STAGING_V_df)

        print()

In [22]:
from datetime import timedelta

df_down = df_down_full
df_up = df_up_full

_processed_customers = []
verbose = False

_matching_process_log_time = []
_matching_process_log_event = []

_direct_customer = []
_direct_match = []
_direct_draw_up_date = []

_multiple_customer = []
_multiple_matches = []
_multiple_drop_dates = []

_no_match_customer = []

save_every_n = 50
to_save_counter = 0
print_every_n = 100

print(len(df_down), "filtered down rows")
print(len(df_up), "filtered up rows")

_customers = []

t0 = time.time()

for index, row in df_down.iterrows():

    customer = row['CUSTOMER']
    draw_down_date = row['DRAW_DOWN_DATE']
    active_card_max = row['ACTIVE_CARD_MAX']

    c = Draw_Down_Customer(customer, draw_down_date, active_card_max)

    _customers.append(c)

idx = 0

_matching_process_log_time.append(str(pd.Timestamp.now()))
_matching_process_log_event.append(" processing range " + str(len(_customers)) + " Draw Down Customers")
do_save_log(_matching_process_log_time, _matching_process_log_event)


186871 filtered down rows
330570 filtered up rows
1 rows successfully written (xzCE9exsko)



In [0]:
idx=0
for c in _customers:

    idx+=1
    
    date_start = pd.to_datetime(c.DRAW_DOWN_DATE) +timedelta(days=-120)
    date_end = pd.to_datetime(c.DRAW_DOWN_DATE) +timedelta(days=120)

    card_delta = c.ACTIVE_CARD_MAX * 0.5
    card_start = c.ACTIVE_CARD_MAX - card_delta
    card_end = c.ACTIVE_CARD_MAX + card_delta

    df_up = df_up_full[(df_up_full.ACTIVE_CARD_MAX>=card_start)&
                   (df_up_full.ACTIVE_CARD_MAX<=card_end)&
                    (df_up_full.DRAW_UP_DATE >= pd.to_datetime(date_start))&
                  (df_up_full.DRAW_UP_DATE <= pd.to_datetime(date_end))].copy()
    
    df_up['distance'] = df_up.apply(lambda x: Levenshtein.ratio(x['CUSTOMER'],c.CUSTOMER),axis=1)
    
    df_up = df_up[df_up.distance>0.8]
    
    for index_up, row_up in df_up.iterrows():

        customer = row_up['CUSTOMER']
        draw_up_date = row_up['DRAW_UP_DATE']
        active_card_max = row_up['ACTIVE_CARD_MAX']

        c.Match_Draw_Up_Customer(customer, draw_up_date, active_card_max)

    if len(c.MATCHING_CUSTOMERS)==1:

        if not c.CUSTOMER in (_processed_customers):

            to_save_counter += 1

            _direct_customer.append(c.CUSTOMER)
            _processed_customers.append(c.CUSTOMER)
            _direct_match.append(c.MATCHING_CUSTOMERS[0])
            _processed_customers.append(c.MATCHING_CUSTOMERS[0])
            _direct_draw_up_date.append(c.DRAW_UP_DATE[0])

            if verbose:
                print()
                print("DIRECT")
                print(c.CUSTOMER, c.WORD_LIST)
                print(c.MATCHING_CUSTOMERS)
                print(c.PERCENT_DIFFERENCE)
                print(c.DAYS_DIFFERENCE)
                print("=====")
                print()

    elif len(c.MATCHING_CUSTOMERS)>1:

        if not c.CUSTOMER in (_processed_customers):

            to_save_counter += 1

            _multiple_customer.append(c.CUSTOMER)
            _processed_customers.append(c.CUSTOMER)
            _multiple_matches.append(c.MATCHING_CUSTOMERS)
            _multiple_drop_dates.append(c.DRAW_UP_DATE)

        if verbose:
            print()
            print("MULTIPLE")
            print(c.CUSTOMER, c.WORD_LIST)
            print(c.MATCHING_CUSTOMERS)
            print(c.PERCENT_DIFFERENCE)
            print(c.DAYS_DIFFERENCE)
            print("=====")
            print()

    else:

        # could not find a match, remove it from future processing
        _no_match_customer.append(c.CUSTOMER)
        _processed_customers.append(c.CUSTOMER)

    if to_save_counter>=save_every_n:

        _matching_process_log_time.append(str(pd.Timestamp.now()))
        _matching_process_log_event.append("writing datasets to snowflake")
        do_save_log(_matching_process_log_time, _matching_process_log_event)

        do_save_direct_matches(_direct_customer, _direct_match, _direct_draw_up_date)
        do_save_multiple_matches(_multiple_customer, _multiple_matches, _multiple_drop_dates)

        _matching_process_log_time.append(str(pd.Timestamp.now()))
        _matching_process_log_event.append("saved " + str(to_save_counter) + " records to snowflake.")
        do_save_log(_matching_process_log_time, _matching_process_log_event)

        to_save_counter = 0

    t1 = time.time()

    avg_duration = (((t1-t0)/idx)/60.0)

    if idx % print_every_n == 0:
        idx_remaining = len(_customers)-idx
        print("processing", idx, "current record:", c.CUSTOMER, ",", idx_remaining, "remaining")
        print(round(avg_duration,2), "avg mins per iteration",  round((avg_duration*idx_remaining)/60,2), "estimated hrs remaining")
        print(len(_direct_customer), "direct match records", len(_multiple_customer), "multiple match records", len(_no_match_customer), "no match records")
        print()

_matching_process_log_time.append(str(pd.Timestamp.now()))
_matching_process_log_event.append("writing datasets to snowflake")
do_save_log(_matching_process_log_time, _matching_process_log_event)

do_save_direct_matches(_direct_customer, _direct_match, _direct_draw_up_date)
do_save_multiple_matches(_multiple_customer, _multiple_matches, _multiple_drop_dates)

_matching_process_log_time.append(str(pd.Timestamp.now()))
_matching_process_log_event.append("saved " + str(to_save_counter) + " records to snowflake.")
do_save_log(_matching_process_log_time, _matching_process_log_event)

processing 100 current record: 1 SOUTH BLUE LLC , 186771 remaining
0.02 avg mins per iteration 55.13 estimated hrs remaining
2 direct match records 0 multiple match records 98 no match records

processing 200 current record: 101821-AUSTIN INDUSTRIES , 186671 remaining
0.01 avg mins per iteration 37.82 estimated hrs remaining
4 direct match records 0 multiple match records 196 no match records

processing 300 current record: 1112232 ONT INC L+M CONTRACTING SALI , 186571 remaining
0.01 avg mins per iteration 32.62 estimated hrs remaining
4 direct match records 0 multiple match records 296 no match records

processing 400 current record: 1248546 ONTARIO INC , 186471 remaining
0.01 avg mins per iteration 30.63 estimated hrs remaining
5 direct match records 0 multiple match records 395 no match records

processing 500 current record: 1412741 ON INC O/A CAMPBELL MOVING , 186371 remaining
0.01 avg mins per iteration 28.86 estimated hrs remaining
6 direct match records 0 multiple match records

processing 4100 current record: A F D MOVERS INC , 182771 remaining
0.01 avg mins per iteration 21.41 estimated hrs remaining
75 direct match records 18 multiple match records 4007 no match records

processing 4200 current record: A I D SECURITY INC , 182671 remaining
0.01 avg mins per iteration 21.35 estimated hrs remaining
76 direct match records 18 multiple match records 4106 no match records

processing 4300 current record: A L S GROUP USA CORP , 182571 remaining
0.01 avg mins per iteration 21.4 estimated hrs remaining
77 direct match records 18 multiple match records 4205 no match records

processing 4400 current record: A MERRIE SWEEPE CHIMNEY SERVIC , 182471 remaining
0.01 avg mins per iteration 21.38 estimated hrs remaining
78 direct match records 18 multiple match records 4304 no match records

processing 4500 current record: A PERSONAL POINT OF VIEW LLC , 182371 remaining
0.01 avg mins per iteration 21.38 estimated hrs remaining
79 direct match records 18 multiple match recor

processing 7900 current record: AERO MANUFACTURING LLC , 178971 remaining
0.01 avg mins per iteration 21.16 estimated hrs remaining
135 direct match records 19 multiple match records 7746 no match records

processing 8000 current record: AFFLUENT EXECUTIVE SIGNING SERVICE LLC , 178871 remaining
0.01 avg mins per iteration 21.16 estimated hrs remaining
138 direct match records 19 multiple match records 7843 no match records

processing 8100 current record: AFFORDABLE PRESSURE WASHING , 178771 remaining
0.01 avg mins per iteration 21.15 estimated hrs remaining
139 direct match records 19 multiple match records 7942 no match records

processing 8200 current record: AG AUTOMOTIVE GROUP LLC , 178671 remaining
0.01 avg mins per iteration 21.16 estimated hrs remaining
141 direct match records 19 multiple match records 8039 no match records

processing 8300 current record: AGGRESSIVE APPLIANCES  FINE FURNITURE INC , 178571 remaining
0.01 avg mins per iteration 21.13 estimated hrs remaining
142

processing 11800 current record: ALTUS IG REAL ESTATE LLC , 175071 remaining
0.01 avg mins per iteration 20.84 estimated hrs remaining
208 direct match records 21 multiple match records 11568 no match records

processing 11900 current record: ALWAYS WITH US CHARITIES AWU CHARITIES , 174971 remaining
0.01 avg mins per iteration 20.82 estimated hrs remaining
209 direct match records 21 multiple match records 11667 no match records

processing 12000 current record: AMAEZING NOVELTIES LLC , 174871 remaining
0.01 avg mins per iteration 20.79 estimated hrs remaining
212 direct match records 21 multiple match records 11764 no match records

processing 12100 current record: AMAZING LLOYD MEDICAL TRANSPORTATION , 174771 remaining
0.01 avg mins per iteration 20.78 estimated hrs remaining
215 direct match records 21 multiple match records 11861 no match records

processing 12200 current record: AMC , 174671 remaining
0.01 avg mins per iteration 20.76 estimated hrs remaining
219 direct match recor

processing 15500 current record: ARIZONA C D J R SHOW LOW , 171371 remaining
0.01 avg mins per iteration 20.37 estimated hrs remaining
325 direct match records 22 multiple match records 15148 no match records

14 rows successfully written (nILuZEPquL)


saving 328 1-1 matching records

328 rows successfully written (oB57m2WrHo)


saving 22 1-n matching records

22 rows successfully written (cXaCOHoGXH)

15 rows successfully written (4LB3CDTCE9)

processing 15600 current record: ARKANSAS PORTABLE TOILETS INC , 171271 remaining
0.01 avg mins per iteration 20.38 estimated hrs remaining
328 direct match records 22 multiple match records 15245 no match records

processing 15700 current record: ARMONK GARAGE INC , 171171 remaining
0.01 avg mins per iteration 20.38 estimated hrs remaining
330 direct match records 22 multiple match records 15343 no match records

processing 15800 current record: AROD PLUMBING  GAS , 171071 remaining
0.01 avg mins per iteration 20.35 estimated hrs remaining
331

processing 19300 current record: BACKHOE RENTAL SERVICES , 167571 remaining
0.01 avg mins per iteration 19.96 estimated hrs remaining
405 direct match records 28 multiple match records 18861 no match records

processing 19400 current record: BAE BEAUTIQUE LLC , 167471 remaining
0.01 avg mins per iteration 19.95 estimated hrs remaining
413 direct match records 29 multiple match records 18952 no match records

processing 19500 current record: BAINBRIDGE OUTDOOR DESIGN BUIL , 167371 remaining
0.01 avg mins per iteration 19.96 estimated hrs remaining
416 direct match records 29 multiple match records 19049 no match records

processing 19600 current record: BAL PERAZIM INTERDENOMINATIONAL CHURCH CHRISTIAN CENTER , 167271 remaining
0.01 avg mins per iteration 19.94 estimated hrs remaining
417 direct match records 29 multiple match records 19148 no match records

18 rows successfully written (yR4X2uG5ql)


saving 421 1-1 matching records

421 rows successfully written (ZWhn3PiY28)


saving 29

processing 23000 current record: BEYOND ELECTRIC LLC , 163871 remaining
0.01 avg mins per iteration 19.64 estimated hrs remaining
509 direct match records 34 multiple match records 22451 no match records

processing 23100 current record: BH WARNE CONSTRUCTION INC , 163771 remaining
0.01 avg mins per iteration 19.63 estimated hrs remaining
513 direct match records 34 multiple match records 22547 no match records

processing 23200 current record: BIG AL CARRIERS LLC , 163671 remaining
0.01 avg mins per iteration 19.62 estimated hrs remaining
515 direct match records 34 multiple match records 22645 no match records

22 rows successfully written (2xVGQXzVPF)


saving 516 1-1 matching records

516 rows successfully written (RYHoT2q3dh)


saving 34 1-n matching records

34 rows successfully written (r5MbYKl9h6)

23 rows successfully written (2AbnWARVZm)

processing 23300 current record: BIG DREAMS LAWN  LANDSCAPES , 163571 remaining
0.01 avg mins per iteration 19.64 estimated hrs remaining
5

processing 26600 current record: BOYS  GIRLS CLUBS OF , 160271 remaining
0.01 avg mins per iteration 19.32 estimated hrs remaining
615 direct match records 37 multiple match records 25939 no match records

processing 26700 current record: BRACKENS CO OP LLC , 160171 remaining
0.01 avg mins per iteration 19.31 estimated hrs remaining
617 direct match records 37 multiple match records 26037 no match records

processing 26800 current record: BRADSHAW LOGISTICS LLC , 160071 remaining
0.01 avg mins per iteration 19.3 estimated hrs remaining
618 direct match records 37 multiple match records 26136 no match records

processing 26900 current record: BRAND MAXIMUM INC , 159971 remaining
0.01 avg mins per iteration 19.29 estimated hrs remaining
620 direct match records 37 multiple match records 26234 no match records

processing 27000 current record: BRANT TAXI INC , 159871 remaining
0.01 avg mins per iteration 19.27 estimated hrs remaining
625 direct match records 37 multiple match records 2632